## 단어 표현(Word Representation) 방법

단어의 표현 방법은 크게 `국소 표현(Local Representation)` 방법과 `분산 표현(Distributed Representation)` 방법으로 나뉨

국소 표현(Local Representation) 방법은 해당 단어 그 자체만 보고, 특정값을 맵핑하여 단어를 표현하는 방법, 이산 표현(Discrete Representation) 이라고도 함

분산 표현(Distributed Representation) 방법은 해당 단어를 표현하고자 주변을 참고하여 단어를 표현하는 방법, 연속 표현(Continuous Represnetation) 이라고도 함

예를 들어 puppy, cute, lovely라는 단어가 있을 때

각 단어에 1번, 2번, 3번 등과 같은 숫자를 맵핑(mapping)하여 부여한다면 이는 국소 표현 방법

puppy라는 단어 근처에는 주로 cute, lovely이라는 단어가 자주 등장하므로, puppy라는 단어에 cute, lovely를 부여한다면 분산 표현 방법

Word Representation
> Local Representation
>> One-hot Vector

>> N-gram

>> Count Based
>>> Bag of Words(DTM)

> Distributed Representation
>> Prediction Based
>>> Word2Vec(FastText)

>> Count Based
>>> Full Document
>>>> LSA

>>> Windows
>>>> Glove

## 백 오브 워즈(Bag of Words)

단어들의 `순서는 전혀 고려하지 않고`, 단어들의 `출현 빈도(frequency)`에만 집중하는 텍스트 데이터의 수치화 표현 방법

BoW를 만드는 과정
1. 단어 집합을 만들어 각 단어에 고유한 정수 인덱스를 부여
2. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터 생성

In [54]:
from konlpy.tag import Okt

okt = Okt()

In [55]:
def build_bag_of_words(document):
    # 온점 제거 및 형태소 분석
    document = document.replace('.', '')
    tokenized_document = okt.morphs(document)

    word_to_index = {}
    bow = []

    for word in tokenized_document:  
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)  
            # BoW에 전부 기본값 1을 넣는다.
            bow.insert(len(word_to_index) - 1, 1)
        else:
            # 재등장하는 단어의 인덱스
            index = word_to_index.get(word)
            # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다.
            bow[index] = bow[index] + 1

    return word_to_index, bow

In [56]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [57]:
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'
vocab, bow = build_bag_of_words(doc2)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector : [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [58]:
# 문서3의 단어 집합은 문서1과 문서2의 단어들을 모두 포함
doc3 = doc1 + ' ' + doc2
vocab, bow = build_bag_of_words(doc3)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
bag of words vector : [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


BoW는 종종 여러 문서의 단어 집합을 합친 뒤에, 해당 단어 집합에 대한 각 문서의 BoW를 구하기도 함

BoW는 각 단어가 등장한 횟수를 수치화하는 텍스트 표현 방법

주로 어떤 단어가 얼마나 등장했는지를 기준으로 문서가 어떤 성격의 문서인지를 판단하는 작업에 사용됨

분류 문제나 여러 문서 간의 유사도를 구하는 문제에 주로 사용됨

사이킷 런에서는 단어의 빈도를 Count하여 Vector로 만드는 CountVectorizer 클래스를 지원

영어에 대해서는 손쉽게 BoW를 생성 가능

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector :', vector.fit_transform(corpus).toarray()) 
# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary :',vector.vocabulary_)

bag of words vector : [[1 1 2 1 2 1]]
vocabulary : {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


알파벳 I는 BoW를 만드는 과정에서 사라졌는데, 이는 CountVectorizer가 기본적으로 길이가 2이상인 문자에 대해서만 토큰으로 인식하기 때문

영어에서는 길이가 짧은 문자를 제거하는 것 또한 전처리 작업으로 고려됨

주의할 것은 CountVectorizer는 단지 띄어쓰기만을 기준으로 단어를 자르는 낮은 수준의 토큰화를 진행하고 BoW를 만든다는 점

이는 영어에서는 문제가 없지만 한국어에 CountVectorizer를 적용하면, 조사 등의 이유로 제대로 BoW가 만들어지지 않음을 의미

자연어 처리에서 별로 의미를 갖지 않는 단어인 불용어를 제거하는 것은

자연어 처리의 정확도를 높이기 위해서 선택할 수 있는 전처리 기법

In [60]:
from nltk.corpus import stopwords

In [61]:
# 불용어 사용
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [62]:
# CountVectorizer 자체 지원하는 불용어 사용
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english")
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}


In [63]:
# NLTK 자체 지원 불용어 사용
text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words=stop_words)
print('bag of words vector :',vect.fit_transform(text).toarray()) 
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


## 문서 단어 행렬(Document-Term Matrix, DTM)

`문서 단어 행렬(Document-Term Matrix, DTM)`이란 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것

`행과 열을 반대`로 선택하면 `TDM`이라고 부르기도 함

각 문서에 대한 BoW를 하나의 행렬로 만든 것으로 BoW 표현을 다수의 문서에 대해서 행렬로 표현하고 부르는 용어

DTM은 매우 간단하고 구현하기도 쉽지만, 본질적으로 가지는 몇 가지 한계가 존재

- 희소 표현(Sparse representation)

DTM 또한 원-핫 벡터와 마찬가지로 공간적 낭비와 계산 리소스를 증가시킬 수 있음

전체 코퍼스가 방대한 데이터라면 문서 벡터의 차원은 수만 이상의 차원을 가질 수도 있고, 문서 벡터가 대부분의 값이 0을 가질 수 있음

대부분의 값이 0인 표현을 희소 벡터(sparse vector) 또는 희소 행렬(sparse matrix)이라 함

희소 벡터는 많은 양의 저장 공간과 높은 계산 복잡도를 요구하고, 전처리를 통해 단어 집합의 크기를 줄이는 일은 BoW 표현을 사용하는 모델에서 중요할 수 있음

- 단순 빈도 수 기반 접근

모든 단어에 대해서 빈도 표기를 하는 방법의 근본적인 문제가 있음

영어에 대해서 DTM을 만들었을 때, 불용어인 the는 어떤 문서이든 자주 등장할 수 밖에 없고, the가 빈도수가 높다고 해서 이 문서들이 유사한 문서라고 판단할 수 없음

각 문서에는 중요한 단어와 불필요한 단어들이 혼재되어 있고, 불용어처럼 빈도수는 높지만 의미를 갖지 못하는 단어가 있을 수 있음

이에 대해 각 단어별 가중치를 준 방법인 TF-IDF 가 있음

## TF-IDF(Term Frequency-Inverse Document Frequency)

TF-IDF(Term Frequency-Inverse Document Frequency)는 단어의 빈도와 역 문서 빈도를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법

DTM을 만든 후, TF-IDF 가중치를 부여

TF-IDF는 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있음

TF-IDF는 TF와 IDF를 곱한 값을 의미

문서를 d, 단어를 t, 문서의 총 개수를 n이라고 표현할 때 TF, DF, IDF는 각각 다음과 같이 정의할 수 있음

tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수

df(t) : 특정 단어 t가 등장한 문서의 수

idf(t) : df(t)에 반비례하는 수

$ idf(t) = log( \frac{n}{1 + df(t)} ) $

idf의 경우 단순 역수를 취하면 총 문서의 수 n이 커질 수록, IDF의 값은 기하급수적으로 증가하므로 log를 사용함

불용어 등과 같이 자주 쓰이는 단어들은 비교적 자주 쓰이지 않는 단어들보다 최소 수십 배 자주 등장하는데, 비교적 자주 쓰이지 않는 단어들조차 희귀 단어들과 비교하면 또 최소 수백 배는 더 자주 등장하는 편

log를 씌워주지 않으면, 희귀 단어들에 엄청난 가중치가 부여될 수 있음

log 안의 식에서 분모에 1을 더해주는 이유도 특정 단어가 전체 문서에서 등장하지 않을 경우에 분모가 0이 되는 상황을 방지하기 위함

TF-IDF은 주로 로그의 밑으로 자연상수 e를 사용하기에 $ idf(t) = ln( \frac{n}{1 + df(t)} ) $ 으로 표현 할 수도 있음

In [64]:
import pandas as pd
from math import log

In [65]:
docs = [
    'you know I want your love',
    'I like you',
    'what should I do ',
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
N = len(docs) 

In [66]:
def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [67]:
# tf(DTM) 계산
result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,I,do,know,like,love,should,want,what,you,your
0,1,0,1,0,1,0,1,0,2,1
1,1,0,0,1,0,0,0,0,1,0
2,1,1,0,0,0,1,0,1,0,0


In [68]:
# idf 계산
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
I,-0.287682
do,0.405465
know,0.405465
like,0.405465
love,0.405465
should,0.405465
want,0.405465
what,0.405465
you,0.000000
your,0.405465


In [69]:
# tf-idf 계산
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,I,do,know,like,love,should,want,what,you,your
0,-0.287682,0.000000,0.405465,0.000000,0.405465,0.000000,0.405465,0.000000,0.0,0.405465
1,-0.287682,0.000000,0.000000,0.405465,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,-0.287682,0.405465,0.000000,0.000000,0.000000,0.405465,0.000000,0.405465,0.0,0.000000


실제로는 대부분의 패키지에서 조정된 식을 사용

df(t)가 n-1 크기일때, idf(d,t)가 0이 되고 더 이상 가중치의 역할을 수행하지 못함을 의미

In [70]:
# 사이킷런을 사용한 TF-IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [71]:
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [72]:
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
